imports

In [1]:
import tensorflow as tf
#import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


2024-10-13 18:28:39.568339: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Helper and preprocessing functions

In [2]:
def tf_scale_imgs(imgs, scale_factor):
    return imgs * scale_factor + 0.5 * (1 - scale_factor) * tf.ones(imgs.shape)


def tf_scale_labels(labels, scale_factor):
    return labels * scale_factor + 0.5 * (1 - scale_factor) * tf.ones(labels.shape)


def tf_f_inv(x, act_fn):
    """ (activation_size, batch_size) """
    if act_fn == "LINEAR":
        m = x
    elif act_fn == "TANH":
        num = tf.ones_like(x) + x
        div = tf.ones_like(x) - x + 1e-7
        m = 0.5 * tf.math.log(num / div)
    elif act_fn == "LOGSIG":
        div = tf.ones_like(x) - x + 1e-7
        m = tf.math.log((x / div) + 1e-7)
    else:
        raise ValueError(f"{act_fn} not supported")
    return m


def boring_movie(x, y, n_steps=4):
  """Stacks a Tensor to create a 'time series' of repeating images."""
  x = tf.stack([x for _ in range(n_steps)], axis=1)  # make the "boring" movie of subsequently following images
  y = tf.stack([y for _ in range(n_steps)], axis=1)
  return x, y


def random_omissions(x, y, omissions=0.5):
  """Randomly omits a fraction of the images in a batch.""" # why needed? -> not needed here 
  mask = tf.random.uniform(tf.shape(x)[:2], 0, 1)[:, :, None, None, None] > omissions
  x = x * tf.cast(mask, dtype=tf.float32)
  return x, y


def img_preproc(x, y, dtype=tf.float32): # add: tf.image.resize(image, [28,28])
  """Cast input image to a certain tf dtype and normalize them between 0 and 1."""
  x = tf.cast(x, dtype) / 255.
  #x = tf_scale_imgs(x, cf.img_scale)
  #y = tf_scale_labels(y, cf.label_scale)
  #x = tf_f_inv(x, "TANH")
  #y = tf.one_hot(y, depth=10)
  return x, y


def flatten(x, y):
  #flattens a video image series (or batch of images) to (n_batch, n_steps, 1) d.
  shape = tf.shape(x)
  if len(shape) == 5: # hack, determining if it's a video or not
    x = tf.reshape(x, [shape[0], shape[1], -1])
  elif len(shape) == 4:
    x = tf.reshape(x, [shape[0], -1])
  return x, y


build the network

In [3]:

class CustomDense(tf.keras.layers.Dense):
    def call(self, inputs):
        """This works like a dense, except for the activation being called earlier."""
        # Apply the activation to the input first
        activated_input = self.activation(inputs)
        # Perform the matrix multiplication and add the bias
        output = tf.matmul(activated_input, self.kernel)
        if self.use_bias:
            output = output + self.bias
        return output


class PredictiveCodingNetwork(tf.keras.Sequential):
    def __init__(self, layers, vars, beta, **kwargs):
        """Initialize a PredictiveCodingNetwork"""
        super().__init__(layers, **kwargs)
        self.vars = tf.convert_to_tensor(vars, dtype=tf.float32)
        self.beta = beta

    def call_with_states(self, x):
        """Note: while model call, call with states and model evaluate take
        2D input, train_step and infer take stacked 3D inputs."""
        x_list = [x]
        for layer in self.layers:
            x = layer(x)
            x_list.append(x)
        return x_list

    def train_step(self, data):
        """Note: while model call, call with states and model evaluate take
        2D input, train_step and infer take stacked 3D inputs."""
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        # do the stuff we do in train_epochs
        outputs, errors = self.infer(x, y)
        self.update_params(outputs, errors)

        # Update metrics (includes the metric that tracks the loss)
        pred = self.call(x[:, 0])
        for metric in self.metrics:
            metric.update_state(y[:, 0], pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def infer(self, x_batch, y_batch):
        """Note: while model call, call with states and model evaluate take
        2D input, train_step and infer take stacked 3D inputs."""
        errors = [None for _ in range(len(self.layers))]
        f_x_arr = [None for _ in range(len(self.layers))]
        f_x_deriv_arr = [None for _ in range(len(self.layers))]
        shape = x_batch.shape
        batch_size, n_steps = shape[0], shape[1]

        for itr in range(n_steps):
            # always update the current forward call
            x_forward_cur = self.call_with_states(x_batch[:, itr])
            x_forward_cur[-1] = y_batch[:, itr]
            # if its the first itr, set x to the current forward call
            if itr == 0:
                x = x_forward_cur
            if itr != 0:
                # update g and x only for consecutive iterations
                for l in range(1, len(self.layers)):
                    g = tf.multiply(tf.matmul(errors[l], self.layers[l].kernel, transpose_b=True), f_x_deriv_arr[l])
                    x[l] = x[l] + (x_forward_cur[l] - x_forward_prev[l]) + self.beta * (-errors[l-1] + g)
            # update f_x etc for every iteration
            for l in range(len(self.layers)):
                f_x = self.layers[l].activation(x[l])
                f_x_deriv_fn = self.get_activation_derivative(self.layers[l].activation)
                f_x_deriv = f_x_deriv_fn(x[l])
                f_x_arr[l] = f_x
                f_x_deriv_arr[l] = f_x_deriv
                errors[l] = (x[l + 1] - tf.matmul(f_x, self.layers[l].kernel) - self.layers[l].bias) / self.vars[l]
            # fill the old forward pass with the current forward pass
            x_forward_prev = x_forward_cur
        return x, errors

    def update_params(self, x, errors):
        """Update the model parameters."""
        batch_size = tf.cast(tf.shape(x[0])[0], tf.float32)
        gradients = []
        for l, layer in enumerate(self.layers):
            grad_w = self.vars[-1] * (1 / batch_size) * tf.matmul(tf.transpose(self.layers[l].activation(x[l])), errors[l])
            grad_b = self.vars[-1] * (1 / batch_size) * tf.reduce_sum(errors[l], axis=0)
            gradients.append((-grad_w, layer.kernel))
            gradients.append((-grad_b, layer.bias))
        self.optimizer.apply_gradients(gradients)

    def get_activation_derivative(self, activation):
        """Return a function for the derivative of the given activation function."""
        activation_fn = tf.keras.activations.get(activation)
        if activation_fn == tf.keras.activations.linear:
            return lambda x: tf.ones_like(x)
        elif activation_fn == tf.keras.activations.tanh:
            return lambda x: 1 - tf.square(tf.nn.tanh(x))
        elif activation_fn == tf.keras.activations.sigmoid:
            return lambda x: tf.nn.sigmoid(x) * (1 - tf.nn.sigmoid(x))
        else:
            raise ValueError(f"{activation} not supported")


Load the data

In [4]:
# Build dataset

# 9 categories: Church, barn  beach  castle  cave  conference_room  forest  library  restaurant
# leading: Barn, beach, library, restaurant, cave 
# trailing: Church, conference room, library, forest

img_dir_lead = '/scratch/guetlid95/datasets/mcdermott_2024/Stimuli_2/Leading/'
img_dir_trail = '/scratch/guetlid95/datasets/mcdermott_2024/Stimuli_2/Trailing/' 
class_names_L = ['barn', 'beach', 'cave', 'library', 'restaurant']
class_names_T = ['castle', 'Church', 'conference_room', 'forest'] # changed the order 
batch_size = 32
image_size = (28,28)
validation_split = 0.1
seed = 123

data_leading = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_lead, 
    label_mode = 'int',
    class_names= class_names_L,
    color_mode = 'rgb',
    image_size = image_size, 
    shuffle = True, 
    #seed = seed
)

data_trailing = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_trail, 
    label_mode = 'int',
    class_names= class_names_T,
    color_mode = 'rgb', 
    image_size = image_size, 
    shuffle = True, 
    #seed = seed
)


NotFoundError: Could not find directory /scratch/guetlid95/datasets/mcdermott_2024/Stimuli_2/Leading/

In [16]:
# Create a leading and trailing dataset with tensorflow

img_dir_lead = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Leading/'
img_dir_trail = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Trailing/'
img_dir_test_lead = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Test/Test_Leading/'
img_dir_test_trail = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Test/Test_Trailing/'
class_names_L = ['barn', 'beach', 'cave', 'library', 'restaurant']
class_names_T = ['castle', 'Church', 'conference_room', 'forest'] # changed the order 
batch_size = None # adjust if needed, e.g., 32
image_size = (28,28)
validation_split = 0.1
seed = 123

data_leading = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_lead, 
    label_mode = 'int',
    class_names= class_names_L,
    batch_size = batch_size,
    color_mode = 'rgb',
    image_size = image_size, 
    #shuffle = True, 
    seed = seed
)

data_trailing = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_trail, 
    label_mode = 'int',
    class_names= class_names_T,
    batch_size = batch_size,
    color_mode = 'rgb', 
    image_size = image_size, 
    #shuffle = True, 
    seed = seed
)

# Check class names and corresponding labels
print("Leading imgs categories:", data_leading.class_names)
print("Trailing imgs categories:", data_trailing.class_names)
   
class_names_dL = data_leading.class_names
for index, class_name in enumerate(class_names_dL):
    print(f"Category: '{class_name}'; Label: {index}")
        
class_names_dT = data_trailing.class_names
for index, class_name in enumerate(class_names_dT ):
    print(f"Category: '{class_name}'; Label: {index}")
    

test_leading = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_test_lead, 
    label_mode = 'int',
    class_names= class_names_L,
    batch_size = batch_size,
    color_mode = 'rgb',
    image_size = image_size, 
    #shuffle = True, 
    seed = seed
)

test_trailing = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_test_trail, 
    label_mode = 'int',
    class_names= class_names_T,
    batch_size = batch_size,
    color_mode = 'rgb', 
    image_size = image_size, 
    #shuffle = True, 
    seed = seed
)
    
print("Leading imgs categories:", test_leading.class_names)
print("Trailing imgs categories:", test_trailing.class_names)
   
class_names_dL = data_leading.class_names
for index, class_name in enumerate(class_names_dL):
    print(f"Category: '{class_name}'; Label: {index}")
        
class_names_dT = data_trailing.class_names
for index, class_name in enumerate(class_names_dT ):
    print(f"Category: '{class_name}'; Label: {index}")

Found 1616 files belonging to 5 classes.
Found 1616 files belonging to 4 classes.
Leading imgs categories: ['barn', 'beach', 'cave', 'library', 'restaurant']
Trailing imgs categories: ['castle', 'Church', 'conference_room', 'forest']
Category: 'barn'; Label: 0
Category: 'beach'; Label: 1
Category: 'cave'; Label: 2
Category: 'library'; Label: 3
Category: 'restaurant'; Label: 4
Category: 'castle'; Label: 0
Category: 'Church'; Label: 1
Category: 'conference_room'; Label: 2
Category: 'forest'; Label: 3
Found 180 files belonging to 5 classes.
Found 180 files belonging to 4 classes.
Leading imgs categories: ['barn', 'beach', 'cave', 'library', 'restaurant']
Trailing imgs categories: ['castle', 'Church', 'conference_room', 'forest']
Category: 'barn'; Label: 0
Category: 'beach'; Label: 1
Category: 'cave'; Label: 2
Category: 'library'; Label: 3
Category: 'restaurant'; Label: 4
Category: 'castle'; Label: 0
Category: 'Church'; Label: 1
Category: 'conference_room'; Label: 2
Category: 'forest'; Lab

In [5]:
# Create a leading and trailing dataset with tensorflow

img_dir_lead = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Leading/'
img_dir_trail = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Trailing/'
img_dir_test_lead = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Test/Test_Leading/'
img_dir_test_trail = '/Users/denisekittelmann/Documents/Python/BiMoL/data/Test/Test_Trailing/'
class_names_L = ['barn', 'beach', 'cave', 'library', 'restaurant']
class_names_T = ['castle', 'Church', 'conference_room', 'forest'] # changed the order 
batch_size = None # adjust if needed, e.g., 32
image_size = (28,28)
validation_split = 0.1
seed = 123

In [4]:
# Create a dict that assigns the correct labels for each leading-trailing img pair

"""
L1 = barn = label 0 
L2 = beach = label 1
L3 = library = label 3
L4 = restaurant = label 4 
L5 = cave = label 2

T6 = Church = label 1   
T7 = conference room = label 2
T8 = castle = label 0   
T9 = forest = label 3

MAPPING:

L1 -> T6 = 0.75 -> (0,1) 
L1 -> T7 = 0.25 -> (0,2)
L1 -> T8 = 0 -> (0,0)
L1 -> T9 = 0 -> (0,3)

L2 -> T6 = 0.75 -> (1,1) 
L2 -> T7 = 0.25 -> (1,2)
L2 -> T8 = 0 -> (1,0)
L2 -> T9 = 0 -> (1,3)

L3 -> T6 = 0 -> (3,1) 
L3 -> T7 = 0 -> (3,2)
L3 -> T8 = 0.5 -> (3,0)
L3 -> T9 = 0.5 -> (3,3)

L4 -> T6 = 0.25 -> (4,1) 
L4 -> T7 = 0.75 -> (4,2)
L4 -> T8 = 0 -> (4,0)
L4 -> T9 = 0 -> (4,3)

L5 -> T6 = 0.25 -> (2,1) 
L5 -> T7 = 0.75 -> (2,2)
L5 -> T8 = 0 -> (2,0)
L5 -> T9 = 0 -> (2,3)

"""

label_dict = {
    (0, 1): 0.0,
    (0, 2): 0.0,
    (0, 0): 0.25,
    (0, 3): 0.75,
    (1, 1): 0.0,
    (1, 2): 0.0,
    (1, 0): 0.25,
    (1, 3): 0.75,
    (3, 1): 0.0,
    (3, 2): 0.0,
    (3, 0): 0.25,
    (3, 3): 0.75,
    (4, 1): 0.0,
    (4, 2): 0.0,
    (4, 0): 0.25,
    (4, 3): 0.75,
    (2, 1): 0.0,
    (2, 2): 0.0,
    (2, 0): 0.5,
    (2, 3): 0.5
}


def imgsequence(img_t1, img_t2, label_t1, label_t2, label_dict): #data_t1, data_t2, label_dict, batch_size=None
    """
    Function that stacks images and prepares labels for each batch.
    + first preprocessing function: flattening 
    """
    
    # go through each img and label in the leading & trailing batch 
    #img_t1, label_t1 = next(iter(data_t1)) # next(iter(data_t1.batch(batch_size)))
    #img_t2, label_t2 = next(iter(data_t2))# next(iter(data_t2.batch(batch_size)))
    
    img_t1 = tf.cast(img_t1, dtype=tf.float32)
    img_t2 = tf.cast(img_t2, dtype=tf.float32)
    #print(type(img_t1))
    
    #print("Shape of img_t1:", img_t1.shape)
    #print("Shape of img_t2:", img_t2.shape)
    print("Shape of label_t1:", label_t1.shape)
    print("Shape of label_t2:", label_t2.shape)
    
    
    # stack imgs from both datatsets into sequence of img
    x = tf.stack([img_t1, img_t1, img_t2, img_t2], axis = 1)
    
    #print("Type x:", x.dtype)
    #print("Shape of x after stacking:", x.shape)
    #print("imgs shape before labeling:", img_t1.shape, img_t2.shape)
    
    #img_t1_flat = tf.reshape(img_t1, [tf.shape(img_t1)[0], -1])  # (batch_size, flattened_image_size)
    #img_t2_flat = tf.reshape(img_t2, [tf.shape(img_t2)[0], -1])  # (batch_size, flattened_image_size)
    #print("img flat:", img_t1_flat.shape)
    
    # Stack the flattened images into sequences (keeping batch intact)
    #x = tf.stack([img_t1_flat, img_t1_flat, img_t2_flat, img_t2_flat], axis=1)
    #print("Shape of x after stacking:", x.shape)
    
    #y = [label_dict.get((int(img_t1), int(img_t2)), 0) for img_t1, img_t2 in zip(label_t1, label_t2)]    
    #print("Current labels:", label_t1, label_t2)
    #print("Type y before:", y.dtype)
    
    y = []
    for img1, img2 in zip(label_t1, label_t2):
        # Create a label for each time step (4 labels in total for each sequence)
        y.append([label_dict.get((int(img1), int(img2)), 0)] * 4)  # Repeat the label for each time step
    
    # Convert labels to a tensor and reshape to (batch_size, n_steps, 1)
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    y = tf.reshape(y, (batch_size, 4, 1))  # Shape (batch_size, 4, 1)

    return x, y
    
    y = tf.convert_to_tensor(y, dtype=tf.float32)
    
    if y.shape[0] != batch_size:
        raise ValueError(f"Y tensor size mismatch: expected {batch_size}, got {y.shape[0]}")
    
    y = tf.reshape(y, (batch_size, 4, 1))
    #print("Type y:", type(y))
    #print("Shape y :", y.shape)
    #print("Type of x:", type(x), "Shape of x:", x.shape, "dtype of x:", x.dtype)
    #print("Type of y:", type(y), "Shape of y:", y.shape, "dtype of y:", y.dtype)
    
    #print(y)
    #print(x)
    
    return x,y 




In [45]:
# Create a dict that assigns the correct labels for each leading-trailing img pair

"""
L1 = barn = label 0 
L2 = beach = label 1
L3 = library = label 3
L4 = restaurant = label 4 
L5 = cave = label 2

T6 = Church = label 1   
T7 = conference room = label 2
T8 = castle = label 0   
T9 = forest = label 3

MAPPING:

L1 -> T6 = 0.75 -> (0,1) 
L1 -> T7 = 0.25 -> (0,2)
L1 -> T8 = 0 -> (0,0)
L1 -> T9 = 0 -> (0,3)

L2 -> T6 = 0.75 -> (1,1) 
L2 -> T7 = 0.25 -> (1,2)
L2 -> T8 = 0 -> (1,0)
L2 -> T9 = 0 -> (1,3)

L3 -> T6 = 0 -> (3,1) 
L3 -> T7 = 0 -> (3,2)
L3 -> T8 = 0.5 -> (3,0)
L3 -> T9 = 0.5 -> (3,3)

L4 -> T6 = 0.25 -> (4,1) 
L4 -> T7 = 0.75 -> (4,2)
L4 -> T8 = 0 -> (4,0)
L4 -> T9 = 0 -> (4,3)

L5 -> T6 = 0.25 -> (2,1) 
L5 -> T7 = 0.75 -> (2,2)
L5 -> T8 = 0 -> (2,0)
L5 -> T9 = 0 -> (2,3)

"""

label_dict = {
    (0, 1): 0.0,
    (0, 2): 0.0,
    (0, 0): 0.25,
    (0, 3): 0.75,
    (1, 1): 0.0,
    (1, 2): 0.0,
    (1, 0): 0.25,
    (1, 3): 0.75,
    (3, 1): 0.0,
    (3, 2): 0.0,
    (3, 0): 0.25,
    (3, 3): 0.75,
    (4, 1): 0.0,
    (4, 2): 0.0,
    (4, 0): 0.25,
    (4, 3): 0.75,
    (2, 1): 0.0,
    (2, 2): 0.0,
    (2, 0): 0.5,
    (2, 3): 0.5
}


def imgsequence(img_t1, img_t2, label_t1, label_t2, label_dict): #data_t1, data_t2, label_dict, batch_size=None
    """
    Function that stacks images and prepares labels for each batch.
    + first preprocessing function: flattening 
    """
    
    # go through each img and label in the leading & trailing batch 
    #img_t1, label_t1 = next(iter(data_t1)) # next(iter(data_t1.batch(batch_size)))
    #img_t2, label_t2 = next(iter(data_t2))# next(iter(data_t2.batch(batch_size)))
    
    img_t1 = tf.cast(img_t1, dtype=tf.float32)
    img_t2 = tf.cast(img_t2, dtype=tf.float32)
    #print(type(img_t1))
    
    #print("Shape of img_t1:", img_t1.shape)
    #print("Shape of img_t2:", img_t2.shape)
    #print("Shape of label_t1:", label_t1.shape)
    #print("Shape of label_t2:", label_t2.shape)
    
    
    # stack imgs from both datatsets into sequence of img
    x = tf.stack([img_t1, img_t1, img_t2, img_t2], axis = 0)
    
    #print("Type x:", x.dtype)
    #print("Shape of x after stacking:", x.shape)
    #print("imgs shape before labeling:", img_t1.shape, img_t2.shape)
    
    #img_t1_flat = tf.reshape(img_t1, [tf.shape(img_t1)[0], -1])  # (batch_size, flattened_image_size)
    #img_t2_flat = tf.reshape(img_t2, [tf.shape(img_t2)[0], -1])  # (batch_size, flattened_image_size)
    #print("img flat:", img_t1_flat.shape)
    
    # Stack the flattened images into sequences (keeping batch intact)
    #x = tf.stack([img_t1_flat, img_t1_flat, img_t2_flat, img_t2_flat], axis=1)
    #print("Shape of x after stacking:", x.shape)
    
    #y = [label_dict.get((int(img_t1), int(img_t2)), 0) for img_t1, img_t2 in zip(label_t1, label_t2)]    
    #print("Current labels:", label_t1, label_t2)
    #print("Type y before:", y.dtype)
    
   # y = []
    #for img1, img2 in zip(label_t1, label_t2):
        # Create a label for each time step (4 labels in total for each sequence)
    #    y.append([label_dict.get((int(img1), int(img2)), 0)] * 4)  # Repeat the label for each time step
    
    # Convert labels to a tensor and reshape to (batch_size, n_steps, 1)
    #y = tf.convert_to_tensor(y, dtype=tf.float32)
    #y = tf.reshape(y, (-1, 4, 1))  # Shape (batch_size, 4, 1)
    #print("Shape y :", y.shape)
    
    # Generate labels based on the sequence
    #y_sequence = []
    #label_value = label_dict.get((int(label_t1), int(label_t2)), 0)
    #y_sequence = [[label_value]] * 4  # Create four labels, one for each step
    #y = tf.convert_to_tensor(y_sequence, dtype=tf.float32)  # Shape (4, 1)


    # Extract labels from scalars and generate the sequence
    #label_value_t1 = label_dict.get(int(label_t1), 0)  # Convert scalar to int and look up in dictionary
    #label_value_t2 = label_dict.get(int(label_t2), 0)  # Convert scalar to int and look up in dictionary
    #print("original label1:", label_t1)
    #print("original label2:", label_t1)
    #print("labelvalue1:", label_value_t1)
    #print("labelvalue2:",label_value_t2)
    
    
    key_t1 = int(label_t1.numpy())
    key_t2 = int(label_t2.numpy())
    
   # print("Current label pair:", (key_t1, key_t2))
    # print("All keys in label_dict:", list(label_dict.keys()))  # Print all keys to check if they match
    
    # Check if the key exists
    if (key_t1, key_t2) in label_dict:
       label_value = label_dict[(key_t1, key_t2)]
        #print(f"Label value found: {label_value}")
    else:
        print(f"Label pair {(key_t1, key_t2)} not found in label_dict, defaulting to 0")
        label_value = 0  # or temporarily change this to another value to diagnose
    
    
    
    # Generate the label sequence (label_t1, label_t1, label_t2, label_t2)
    y_sequence = [[label_value], [label_value], [label_value], [label_value]]
    y = tf.convert_to_tensor(y_sequence, dtype=tf.float32)  # Shape (4, 1)

    # Print for verification
   # print("Generated labels (y):", y.numpy())
    #print("Shape of y:", y.shape)

    return x, y
    
    #y = tf.convert_to_tensor(y, dtype=tf.float32)
    
    #if y.shape[0] != batch_size:
     #   raise ValueError(f"Y tensor size mismatch: expected {batch_size}, got {y.shape[0]}")
    
    #y = tf.reshape(y, (batch_size, 4, 1))
    #print("Type y:", type(y))
    #print("Shape y :", y.shape)
    #print("Type of x:", type(x), "Shape of x:", x.shape, "dtype of x:", x.dtype)
    #print("Type of y:", type(y), "Shape of y:", y.shape, "dtype of y:", y.dtype)
    
    #print(y)
    #print(x)
    
    #return x,y 


In [46]:
def generate_dataset(img_dirt1, img_dirt2, class_namest1, class_namest2, label_dict, image_size = None, seed = None):
        
    # ADD Randomisation 
    # load datatsets -> dont batch 
    # shuffle both datasets 
    # put them together
    # output: samples 
    
    tf.random.set_seed(seed)
    
    data_t1 = tf.keras.preprocessing.image_dataset_from_directory(
        img_dirt1, 
        label_mode = 'int',
        class_names= class_namest1,
        batch_size = None,
        color_mode = 'rgb',
        image_size = image_size, 
        #shuffle = True, 
        seed = seed
        )

    data_t2 = tf.keras.preprocessing.image_dataset_from_directory(
        img_dirt2, 
        label_mode = 'int',
        class_names= class_namest2,
        batch_size = None,
        color_mode = 'rgb', 
        image_size = image_size, 
        #shuffle = True, 
        seed = seed
    )
    
    data_t1.shuffle(99999)
    data_t2.shuffle(99999)
    
    # iterate through shuffled leading and trailing datasets
    leading = iter(data_t1)
    trailing = iter(data_t2) 
              
    while True:
        try:
            # Retrieve single samples
            img_t1, label_t1 = next(leading)
            img_t2, label_t2 = next(trailing)
            #print("test img shapes.", img_t1.shape, img_t2.shape)
            #print("Shape of label_t1:", label_t1.shape)
            #print("Shape of label_t2:", label_t2.shape)

            # Generate x, y pairs for single samples
            x, y = imgsequence(img_t1, img_t2, label_t1, label_t2, label_dict)
            #print("test x:", x.shape)
            #print("test y:", y.shape) 
            yield x, y
            
        except StopIteration:
            # Break the loop if no more samples
            break
        
        
# Create new dataset 
n_steps = 4
seed = 123
#batch_size = 32
dataset = tf.data.Dataset.from_generator(
    lambda: generate_dataset(img_dir_lead, img_dir_trail, class_names_L, class_names_T, label_dict, image_size = (28,28), seed = seed),
    output_signature=(
        tf.TensorSpec(shape=(n_steps, 28, 28, 3), dtype=tf.float32),  # shape of x 2352; skip batch_size -> dataset.batch()
        tf.TensorSpec(shape=(n_steps, 1), dtype=tf.float32)  # shape of y 
    )
) 

print(dataset)
  
     

<_FlatMapDataset element_spec=(TensorSpec(shape=(4, 28, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4, 1), dtype=tf.float32, name=None))>


In [44]:
for x, y in dataset.batch(32).take(1):
    print("test x:", x.shape)  # Should print (32, 4, 28, 28, 3)
    print("test y:", y.shape) 

Found 1616 files belonging to 5 classes.
Found 1616 files belonging to 4 classes.
Current label pair: (2, 1)
All keys in label_dict: [(0, 1), (0, 2), (0, 0), (0, 3), (1, 1), (1, 2), (1, 0), (1, 3), (3, 1), (3, 2), (3, 0), (3, 3), (4, 1), (4, 2), (4, 0), (4, 3), (2, 1), (2, 2), (2, 0), (2, 3)]
Label value found: 0.0
Generated labels (y): [[0.]
 [0.]
 [0.]
 [0.]]
Current label pair: (4, 3)
All keys in label_dict: [(0, 1), (0, 2), (0, 0), (0, 3), (1, 1), (1, 2), (1, 0), (1, 3), (3, 1), (3, 2), (3, 0), (3, 3), (4, 1), (4, 2), (4, 0), (4, 3), (2, 1), (2, 2), (2, 0), (2, 3)]
Label value found: 0.75
Generated labels (y): [[0.75]
 [0.75]
 [0.75]
 [0.75]]
Current label pair: (0, 0)
All keys in label_dict: [(0, 1), (0, 2), (0, 0), (0, 3), (1, 1), (1, 2), (1, 0), (1, 3), (3, 1), (3, 2), (3, 0), (3, 3), (4, 1), (4, 2), (4, 0), (4, 3), (2, 1), (2, 2), (2, 0), (2, 3)]
Label value found: 0.25
Generated labels (y): [[0.25]
 [0.25]
 [0.25]
 [0.25]]
Current label pair: (3, 2)
All keys in label_dict: [(0

In [42]:
def generate_dataset(data_t1, data_t2, label_dict, batch_size=None):
    
    # iterate through leading and trailing datasets and batch them accordingly 
    leading = iter(data_t1.batch(batch_size))
    trailing = iter(data_t2.batch(batch_size)) # re-check -> sample batch
    
    # ADD Randomisation 
    # load datatsets -> dont batch 
    # shuffle both datasets 
    # put them together
    # output: samples 

    while True:
        try:
            
            img_t1, label_t1 = next(leading)
            img_t2, label_t2 = next(trailing)
            #print("test img shapes.", img_t1.shape, img_t2.shape)

            # Check if both current batches == batch_size, if not stop generating the dataset 
            if img_t1.shape[0] == batch_size and img_t2.shape[0] == batch_size:
                
                x, y = imgsequence(img_t1, img_t2, label_t1, label_t2, label_dict, batch_size) # delete batch 
                #print("test x:", x.shape)
                #print("test y:", y.shape)  
                yield x, y
            else:
                print(f"Skipping batch: leading batch size = {img_t1.shape[0]}, trailing batch size = {img_t2.shape[0]}")
                break
        
        except StopIteration:
            # Break when there are no more samples
            break
        

# Create new dataset 
n_steps = 4
batch_size = 32
dataset = tf.data.Dataset.from_generator(
    lambda: generate_dataset(data_leading, data_trailing, label_dict, batch_size=32),
    output_signature=(
        tf.TensorSpec(shape=(batch_size, n_steps, 28, 28, 3), dtype=tf.float32),  # shape of x 2352; skip batch_size -> dataset.batch()
        tf.TensorSpec(shape=(batch_size, n_steps, 1), dtype=tf.float32)  # shape of y 
    )
)

print(dataset)

#for x, y in dataset:
#    print("Shape of x:", x.shape)
#    print("Shape of y:", y.shape)
#    print("Type of x:", x.dtype)
#    print("Type of y:", y.dtype)
    

<_FlatMapDataset element_spec=(TensorSpec(shape=(32, 4, 28, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 4, 1), dtype=tf.float32, name=None))>


In [ ]:
def generate_dataset(img_dirt1, img_dirt2, class_namest1, class_namest2, image_size = None, seed = None, label_dict):
    
    # iterate through leading and trailing datasets and batch them accordingly 
    leading = iter(data_t1)
    trailing = iter(data_t2) # re-check -> sample batch
    
    # ADD Randomisation 
    # load datatsets -> dont batch 
    # shuffle both datasets 
    # put them together
    # output: samples 
    
    data_t1 = tf.keras.preprocessing.image_dataset_from_directory(
    img_dir_lead, 
    label_mode = 'int',
    class_names= class_names_L,
    batch_size = None,
    color_mode = 'rgb',
    image_size = image_size, 
    #shuffle = True, 
    seed = seed
    )

    data_t2 = tf.keras.preprocessing.image_dataset_from_directory(
        img_dir_trail, 
        label_mode = 'int',
        class_names= class_names_T,
        batch_size = None,
        color_mode = 'rgb', 
        image_size = image_size, 
        #shuffle = True, 
        seed = seed
    )
    
    data_t1.shuffle(99999)
    data_t2.shuffle(99999)
    
    
    
    while True:
        try:
            
            img_t1, label_t1 = next(leading)
            img_t2, label_t2 = next(trailing)
            #print("test img shapes.", img_t1.shape, img_t2.shape)

            # Check if both current batches == batch_size, if not stop generating the dataset 
            if img_t1.shape[0] == batch_size and img_t2.shape[0] == batch_size:
                
                x, y = imgsequence(img_t1, img_t2, label_t1, label_t2, label_dict, batch_size) # delete batch 
                #print("test x:", x.shape)
                #print("test y:", y.shape)  
                yield x, y
            else:
                print(f"Skipping batch: leading batch size = {img_t1.shape[0]}, trailing batch size = {img_t2.shape[0]}")
                break
        
        except StopIteration:
            # Break when there are no more samples
            break
       

In [ ]:
# Get training sequence accuracy

def seq_accuracy(t[batch_size,4], o[batch_size, 4]):
    
    accuracies = []
    
    for i in range(4):
        
        accuracies.append(tf.keras.metrics.accuracy())
        
    return accuracies
        

In [35]:
dataset_size = sum(1 for _ in dataset)
train_size = int(0.9 * dataset_size)
val_size = dataset_size - train_size
print(f"Val_dataset size: {val_size}")

# Split into train and validation datasets
train_dataset = dataset.take(train_size)  # Take the first 90% for training
val_dataset = dataset.skip(train_size).take(val_size)

#print(train_dataset.element_spec)
#print(val_dataset.element_spec)


batch_count = 0
for _ in train_dataset:
    batch_count += 1
print(f"Total number of batches in train dataset: {batch_count}")

batch_count = 0
for _ in val_dataset:
    batch_count += 1
print(f"Total number of batches in val dataset: {batch_count}")

Skipping batch: leading batch size = 4, trailing batch size = 4
Validation dataset size: 6


2024-10-04 14:02:35.988916: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-10-04 14:02:38.175796: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Total number of batches in train dataset: 50
Total number of batches in val dataset: 6


2024-10-04 14:02:40.766395: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [36]:
for batch_x, batch_y in val_dataset.take(1):  # Taking one batch from your dataset
    flattened_x, flattened_y = flatten(batch_x, batch_y)
    print(f"Original shape of x: {batch_x.shape}")
    print(f"Flattened shape of x: {flattened_x.shape}")
    print(f"Shape of y (labels): {flattened_y.shape}")

Original shape of x: (32, 4, 28, 28, 3)
Flattened shape of x: (32, 4, 2352)
Shape of y (labels): (32, 4, 1)


2024-10-04 14:02:44.381066: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


train the network

In [38]:
# define the model
model = PredictiveCodingNetwork([CustomDense(units=500, activation="tanh"),
                                 CustomDense(units=500, activation="tanh"),
                                 CustomDense(units=1, activation="sigmoid")],
                                vars=[1, 1, 1], # variances. This is super useless and in the code only the last variance is used
                                beta=0.1)

model.build([None, 2352])
model.compile(optimizer=tf.keras.optimizers.AdamW(),
              metrics= seq_accuracy(),  #["accuracy"],
              loss="CategoricalCrossentropy",  # This is just a sham loss we need so model.evaluate doesn't throw an error. We don't use it.
              )

# evaluate accuracy and train model
#loss, accuracy = model.evaluate(val_dataset.map(img_preproc).map(flatten))
#print(f"Accuracy before training: {accuracy}")

for loop here 

train_dataset = tf.data.Dataset.from_generator(
    lambda: generate_dataset(data_leading, data_trailing, label_dict, batch_size=32),
    output_signature=(
        tf.TensorSpec(shape=(batch_size, n_steps, 28, 28, 3), dtype=tf.float32),  # shape of x 2352; skip batch_size -> dataset.batch()
        tf.TensorSpec(shape=(batch_size, n_steps, 1), dtype=tf.float32)  # shape of y 
    )
)

model.fit(train_dataset.batch(32).shuffle(99999).map(img_preproc).map(flatten),
          #validation_data=val_dataset.batch(batch_size).map(img_preproc).map(flatten),
          epochs=3)



loss, accuracy = model.evaluate(val_dataset.map(img_preproc).map(flatten))
print(f"Accuracy after training: {accuracy}")



Epoch 1/3
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.5890 - loss: 0.2096
Epoch 2/3


2024-10-04 14:02:57.555334: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.6421 - loss: 0.2307
Epoch 3/3


2024-10-04 14:03:01.454090: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.6544 - loss: 0.1933


2024-10-04 14:03:05.522770: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


ValueError: Exception encountered when calling PredictiveCodingNetwork.call().

[1mInvalid input shape for input Tensor("IteratorGetNext:0", shape=(32, 4, 2352), dtype=float32). Expected shape (None, 2352), but input has incompatible shape (32, 4, 2352)[0m

Arguments received by PredictiveCodingNetwork.call():
  • inputs=tf.Tensor(shape=(32, 4, 2352), dtype=float32)
  • training=False
  • mask=None